# Live Model Server Testing

Test your model server via HTTP calls

```markdown
:param table:         csv/parquet table with test data
:param addr:          function address/url
:param label_column:  name of the label column in table
:param model:         tested model name 
:param match_err:     raise error on validation (require proper test set)
:param rows:          number of rows to use from test set

```

## Deploy model server for testing

In [1]:
import os
import mlrun

project_name = "sk-project"
MODEL_PATH = "https://s3.wasabisys.com/iguazio/models/iris/model.pkl"

_, artifact_path = mlrun.set_environment(
    api_path="http://mlrun-api:8080", artifact_path=os.path.abspath("./")
)

# import model server function from hub
fn = mlrun.import_function("hub://model_server")
fn.add_model("mymodel", MODEL_PATH)
address = fn.deploy()

> 2021-02-16 12:27:05,842 [info] Starting remote function deploy
2021-02-16 12:27:06  (info) Deploying function
2021-02-16 12:27:06  (info) Building
2021-02-16 12:27:06  (info) Staging files and preparing base images
2021-02-16 12:27:06  (info) Building processor image
2021-02-16 12:27:07  (info) Build complete
2021-02-16 12:27:13  (info) Function deploy complete
> 2021-02-16 12:27:14,088 [info] function deployed, address=default-tenant.app.vmdev36.lab.iguazeng.com:32699


In [2]:
user_name = os.getenv("V3IO_USERNAME")

fn.apply(mlrun.mount_v3io())
fn.set_envs(
    {
        "SERVING_MODEL_iris_dataset_v1": MODEL_PATH,
        "INFERENCE_STREAM": "users/{}/tststream".format(user_name),
    }
)

address = fn.deploy()

> 2021-02-16 12:27:14,113 [info] Starting remote function deploy
2021-02-16 12:27:14  (info) Deploying function
2021-02-16 12:27:14  (info) Building
2021-02-16 12:27:14  (info) Staging files and preparing base images
2021-02-16 12:27:14  (info) Building processor image
2021-02-16 12:27:15  (info) Build complete
> 2021-02-16 12:27:25,889 [info] function deployed, address=default-tenant.app.vmdev36.lab.iguazeng.com:32699


## Run model server tester locally

In [3]:
from model_server_tester import model_server_tester

# run the function locally
DATA_PATH = "https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv"

gen = mlrun.run_local(
    name="model_server_tester",
    handler=model_server_tester,
    params={"addr": address, "model": "mymodel"},
    inputs={"table": DATA_PATH},
    project=project_name,
    artifact_path=os.path.join(artifact_path, "data"),
)

> 2021-02-16 12:27:25,915 [info] starting run model_server_tester uid=63c114726b864ec49a037f169dffb473 DB=http://mlrun-api:8080
> 2021-02-16 12:27:26,659 [info] testing with dataset against http://default-tenant.app.vmdev36.lab.iguazeng.com:32699, model: mymodel
> 2021-02-16 12:27:27,601 [info] run 20 tests, 0 errors and 8 match expected value


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...9dffb473,0,Feb 16 12:27:25,completed,model_server_tester,v3io_user=adminkind=handlerowner=adminhost=jupyter-7b854d9bd6-mkmbn,table,addr=http://default-tenant.app.vmdev36.lab.iguazeng.com:32699model=mymodel,total_tests=20errors=0match=8avg_latency=42295min_latency=40016max_latency=48875,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run 63c114726b864ec49a037f169dffb473 --project sk-project , !mlrun logs 63c114726b864ec49a037f169dffb473 --project sk-project
> 2021-02-16 12:27:27,685 [info] run executed, status=completed


## Save

In [4]:
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)

In [5]:
test_func = mlrun.code_to_function(
    name=items["name"],
    kind=items["spec"]["kind"],
    filename=items["spec"]["filename"],
    image=items["spec"]["image"],
    description=items["description"],
    categories=items["categories"],
    labels=items["labels"],
    requirements=items["spec"]["requirements"],
)

test_func.export("model_server_tester.yaml")

> 2021-02-16 12:27:27,817 [info] function spec saved to path: model_server_tester.yaml


## Run remotely

In [6]:
test_func.run(
    mlrun.NewTask(
        name="model_server_tester",
        handler=model_server_tester,
        params={"addr": address, "model": "mymodel"},
        inputs={"table": DATA_PATH},
        project=project_name,
        artifact_path=os.path.join(artifact_path, "data"),
    )
)

> 2021-02-16 12:27:27,835 [info] starting run model_server_tester uid=9c502975c5e44376b6980b1c396e29e5 DB=http://mlrun-api:8080
> 2021-02-16 12:27:28,038 [info] Job is running in the background, pod: model-server-tester-t5tkq
> 2021-02-16 12:27:31,821 [info] testing with dataset against http://default-tenant.app.vmdev36.lab.iguazeng.com:32699, model: mymodel
> 2021-02-16 12:27:32,964 [info] run 20 tests, 0 errors and 6 match expected value
> 2021-02-16 12:27:33,007 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...396e29e5,0,Feb 16 12:27:31,completed,model_server_tester,v3io_user=adminkind=jobowner=adminhost=model-server-tester-t5tkq,table,addr=http://default-tenant.app.vmdev36.lab.iguazeng.com:32699model=mymodel,total_tests=20errors=0match=6avg_latency=49641min_latency=40883max_latency=74428,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run 9c502975c5e44376b6980b1c396e29e5 --project sk-project , !mlrun logs 9c502975c5e44376b6980b1c396e29e5 --project sk-project
> 2021-02-16 12:27:34,217 [info] run executed, status=completed
